# Chp 12: Part 1

In [43]:
# Listing 12.1 Reweighting a probability distribution to a different temperature
# Reweighting a probability distribution to a different temperature

# import numpy as np
# def reweight_distribution(original_distribution, temperature=0.5):
#     distribution = np.log(original_distribution) / temperature
#     distribution = np.exp(distribution)
#     return distribution / np.sum(distribution)

In [44]:
sequence_length = 100
vocab_size = 15_000

def HR():
    # print char * numeric
    print('-' * 80)

def set_mixed_precision():
    import tensorflow as tf
    from tensorflow import keras

    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        print("GPU mode - switch to mixed precision.")
        print("Every layer will use a 16-bit compute dtype and float32 variable dtype by default.")
        keras.mixed_precision.set_global_policy("mixed_float16")

    HR()
    print("global policy:", tf.keras.mixed_precision.global_policy())

set_mixed_precision()

GPU mode - switch to mixed precision.
Every layer will use a 16-bit compute dtype and float32 variable dtype by default.
--------------------------------------------------------------------------------
global policy: <Policy "mixed_float16">


In [45]:
# Implementing text generation with Keras

# Listing 12.2 Downloading and uncompressing the IMDB movie reviews dataset
def listing12_2():
    import os

    dirpath = 'aclImdb'

    if not os.path.isdir(dirpath):
        print(f'{dirpath} not found, creating directory')
        HR()
        !wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
        !tar -xf aclImdb_v1.tar.gz
    else:
        print(f'{dirpath} already exits')

listing12_2()

aclImdb already exits


In [46]:
#  Listing 12.3 Creating a Dataset that yields the content of a set of text files (one file = one sample)

def listing12_3():
    import tensorflow as tf
    from tensorflow import keras

    dataset = keras.preprocessing.text_dataset_from_directory(
        directory="aclImdb", label_mode=None, batch_size=256
    )

    dataset = dataset.map(lambda x: tf.strings.regex_replace(x, "<br />", " "))
    
    return dataset

dataset = listing12_3()

print(dataset)

Found 100006 files belonging to 1 classes.
<MapDataset shapes: (None,), types: tf.string>


**Preparing a TextVectorization layer**

In [51]:
# Listing 12.4 Preparing a TextVectorization layer
# Use a TextVectorization layer to compute the vocabulary we’ll be working 
# with. We’ll only use the first sequence_length words of each review: 
# our TextVectorization layer will cut off anything beyond that when 
# vectorizing a text.

def listing12_4():
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

    

    text_vectorization = tf.keras.layers.experimental.preprocessing.TextVectorization(
        max_tokens=vocab_size,
        output_mode="int",
        output_sequence_length=sequence_length,
    )
    
    text_vectorization.adapt(dataset)

    # Listing 12.5 Setting up a language modeling dataset
    def prepare_lm_dataset(text_batch):
        vectorized_sequences = text_vectorization(text_batch)
        x = vectorized_sequences[:, :-1]
        y = vectorized_sequences[:, 1:]
        return x, y

    lm_dataset = dataset.map(prepare_lm_dataset)
    lm_dataset = lm_dataset.prefetch(8)

    return text_vectorization, lm_dataset

text_vectorization, lm_dataset = listing12_4()

In [52]:
# A Transformer-based sequence-to-sequence model, p.423

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


    

In [53]:
# Listing 12.6 A simple Transformer-based language model
def listing12_6():
    from tensorflow import keras
    from tensorflow.keras import layers

    embed_dim = 256
    latent_dim = 2048
    num_heads = 2

    inputs = keras.Input(shape=(None,), dtype="int64")
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
    x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
    outputs = layers.Dense(vocab_size, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    
    model.compile(
        loss="sparse_categorical_crossentropy", 
        optimizer="rmsprop"
    )
    return model
    
model = listing12_6()

In [55]:
# 12.1.5 A text-generation callback with variable-temperature sampling
# The text generation callback
# p.426

def listing12_7():
    import numpy as np

    tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

    def sample_next(predictions, temperature=1.0):
        predictions = np.asarray(predictions).astype("float64")
        predictions = np.log(predictions) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, predictions, 1)
        return np.argmax(probas)

    class TextGenerator(keras.callbacks.Callback):
        def __init__(self,
                    prompt,
                    generate_length,
                    model_input_length,
                    temperatures=(1.,),
                    print_freq=1):
            self.prompt = prompt
            self.generate_length = generate_length
            self.model_input_length = model_input_length
            self.temperatures = temperatures
            self.print_freq = print_freq

        def on_epoch_end(self, epoch, logs=None):
            if (epoch + 1) % self.print_freq != 0:
                return
            for temperature in self.temperatures:
                print("== Generating with temperature", temperature)
                sentence = self.prompt
                for i in range(self.generate_length):
                    tokenized_sentence = text_vectorization([sentence])
                    predictions = self.model(tokenized_sentence)
                    next_token = sample_next(predictions[0, i, :])
                    sampled_token = tokens_index[next_token]
                    sentence += " " + sampled_token
                print(sentence)

    prompt = "This movie"
    
    text_gen_callback = TextGenerator(
        prompt,
        generate_length=10,
        model_input_length=sequence_length,
        temperatures=(0.2, 0.5, 0.7, 1., 1.5)
    )


    # Fitting the language model
    # This takes a long time
    model.fit(
        lm_dataset, 
        epochs=10, 
        callbacks=[text_gen_callback]
    )

listing12_7()

Epoch 1/10
391/391 [==============================] - 105s 267ms/step - loss: nan
== Generating with temperature 0.2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':


This movie movie was with 15 mastermind [UNK] avoid as a [UNK]
== Generating with temperature 0.5
This movie pathetic am questions and mention tasteless all but reality you
== Generating with temperature 0.7
This movie fictional wasnt very my city leaves military any first kind
== Generating with temperature 1.0
This movie thing another the who end thought and they i not
== Generating with temperature 1.5
This movie one i edge stephen to [UNK] the very the a
Epoch 2/10
391/391 [==============================] - 105s 268ms/step - loss: nan
== Generating with temperature 0.2
This movie by is it [UNK] it write is has not an
== Generating with temperature 0.5
This movie film was are an tom put and  the [UNK]
== Generating with temperature 0.7
This movie is is the the i recent tried ranked gratuitous but
== Generating with temperature 1.0
This movie i is i run guess with i that not the
== Generating with temperature 1.5
This movie was i loved liked the the singer moment in it
Epoch 3/10
391